In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Kampen, the Netherlands?")
print(search_results)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.accuweather.com/en/nl/kampen/250100/weather-forecast/250100', 'content': 'Kampen, Overijssel, Netherlands Weather Forecast, with current conditions, wind, air quality, and what to expect for the next 3 days.'}, {'url': 'https://www.bbc.com/weather/2753106', 'content': 'Observed at 20:00, Saturday 15 June BBC Weather in association with MeteoGroup, external All times are Central European Summer Time (Europe/Amsterdam, GMT+2) unless otherwise stated ...'}]


In [3]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
model_with_tools = model.bind_tools(tools)

In [6]:
from langchain_core.messages import HumanMessage
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Osnabrück, Germany?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")
# Model is just telling us to call a tool

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Osnabrück, Germany'}, 'id': 'call_sGT3RBCY8nvJaX1VlNlobCkP'}]


In [7]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [8]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]
# No tool call

[HumanMessage(content='hi!', id='aeb5bcdf-b82d-4c2d-af41-ce6181cfe791'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f205f6b5-279d-4c21-b8c9-000afb939237-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [9]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="current weather in osnabrueck germany?")]}
)
response["messages"]
# Should call tool

[HumanMessage(content='current weather in osnabrueck germany?', id='1f7c595e-8faa-4977-af2f-8f615bbc73d1'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_L8YZAdbwPsz1Kw8qe6KUgIHg', 'function': {'arguments': '{"query":"current weather in Osnabrueck, Germany"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 91, 'total_tokens': 118}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c1dd667a-5755-4245-a3c3-4d05f35ef592-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Osnabrueck, Germany'}, 'id': 'call_L8YZAdbwPsz1Kw8qe6KUgIHg'}], usage_metadata={'input_tokens': 91, 'output_tokens': 27, 'total_tokens': 118}),
 ToolMessage(content='[{"url": "https://www.accuweather.com/en/de/osnabr\\u00fcck/49074/weather-forecast/169819", "content": "Osnabr\\u00fcck, Lower Saxony, Ge

In [10]:
# streaming messages
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_HmMqnmoHjOkbp51wEFN068Mi', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-aca31f3e-a0a5-450d-9ee0-4d1f1f10089c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_HmMqnmoHjOkbp51wEFN068Mi'}], usage_metadata={'input_tokens': 88, 'output_tokens': 21, 'total_tokens': 109})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.accuweather.com/en/us/san-francisco/94103/june-weather/347629", "content": "Get the monthly weather forecast for San Francisco, CA, including daily high/low, historical averages, to help you plan ah

In [11]:
# streaming tokens
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

C:\Users\wafoe\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'weather in San Francisco'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://www.accuweather.com/en/us/san-francisco/94103/june-weather/347629', 'content': 'Get the monthly weather forecast for San Francisco, CA, including daily high/low, historical averages, to help you plan ahead.'}, {'url': 'https://world-weather.info/forecast/usa/san_francisco/june-2024/', 'content': 'Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ San Francisco Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.info.'}]
--
I| found| some| resources| for| the| weather| in| San| Francisco|:
|1|.| [|Acc|u|Weather|](|https|://|www|.acc|u|weather|.com|/en|/us|/s|an|-fr|anc|isco|/|941|03|/j|une|-|weather|/|347|629|):| Get| the| monthly| weather| forecast| for| San| Francisco|,| including| daily| high|/|low| temperatures| and| historical| aver

In [12]:
# stateful agent
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [13]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi, im julian, please say hello to me!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Julian! It's great to have you here. How can I assist you today?", response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 93, 'total_tokens': 112}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-409c2ab5-c106-4b02-9e9f-f20ccb1be170-0', usage_metadata={'input_tokens': 93, 'output_tokens': 19, 'total_tokens': 112})]}}
----


In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="thanks mate! whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Julian! How can I help you further, Julian?', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 126, 'total_tokens': 141}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9634e246-03d4-4fd6-b818-b4bcb5c35e97-0', usage_metadata={'input_tokens': 126, 'output_tokens': 15, 'total_tokens': 141})]}}
----
